.. _nb_results:

## Result

After an algorithm has been executed a result object is returned. In the following, single- and multi-objective runs with and without constraints are shown and the corresponding `Result` object is explained:

In [1]:
from pymoo.algorithms.so_genetic_algorithm import GA
from pymoo.factory import get_problem
from pymoo.optimize import minimize


problem = get_problem("sphere")
algorithm = GA(pop_size=5)
res = minimize(problem,
               algorithm,
               ('n_gen', 30),
               seed=1)


In this single-objective optimization problem, there exists a single best solution that was found. 
The result contains directly the best found values in the corresponding spaces. 

- `res.X` design space values are 
- `res.F` objective spaces values
- `res.G` constraint values
- `res.CV` aggregated constraint violation
- `res.algorithm` algorithm object
- `res.pop` final population object
- `res.history` history of algorithm object. (only if `save_history` has been enabled during the algorithm initialization)
- `res.time` the time required to run the algorithm

Note, when the `minimize` function is called a deep copy of the algorithm object is created.
This ensures that two independent runs with the same algorithm and same random seed have the same results without any side effects.

The values from the final population can be extracted by using the `get` method. 
The population object is used internally, and store information for each individual. The `get`
method allows to return vectors or matrices based on the provided properties.


In [2]:
res.X, res.F, res.G, res.CV

(array([0.38401994, 0.51096839, 0.51958345, 0.50705362, 0.64625327,
        0.52972181, 0.51346167, 0.57393803, 0.48975355, 0.49213286]),
 array([0.04209328]),
 None,
 array([0.]))

In [3]:
res.algorithm, res.pop

(<pymoo.algorithms.so_genetic_algorithm.GA at 0x122ea4050>,
 Population([<pymoo.model.individual.Individual object at 0x122ea91d0>,
            dtype=object))

The values from the final population can be extracted by using the `get` method. 
The population object is used internally, and store information for each individual. The `get`
method allows to return vectors or matrices based on the provided properties.

In [4]:
res.pop.get("X"), res.pop.get("F")

(array([[0.38401994, 0.51096839, 0.51958345, 0.50705362, 0.64625327,
         0.52972181, 0.51346167, 0.57393803, 0.48975355, 0.49213286],
        [0.38401994, 0.51096839, 0.51958345, 0.50705362, 0.64625327,
         0.52972181, 0.51346167, 0.57393803, 0.51785226, 0.49213286],
        [0.38401994, 0.51099826, 0.44223106, 0.50705362, 0.64625327,
         0.52808581, 0.51346167, 0.57393803, 0.39462856, 0.49213286],
        [0.38401994, 0.51099826, 0.44223106, 0.50705362, 0.64625327,
         0.52864113, 0.51346167, 0.57393803, 0.39462856, 0.49213286],
        [0.38401994, 0.47389685, 0.44223106, 0.50705362, 0.64625327,
         0.52808581, 0.51346167, 0.57393803, 0.39462856, 0.49213286]]),
 array([[0.04209328],
        [0.04230699],
        [0.05595125],
        [0.05598275],
        [0.05651167]]))

In this run, the problem did not have any constraints and therefore `res.G` evaluated to `None`.
Also note, that `res.CV` will always be set to 0 no matter if the problem has constraints or not.

Let us consider a problem that has, in fact, constraints:

In [5]:
problem = get_problem("g01")
algorithm = GA(pop_size=5)
res = minimize(problem,
               algorithm,
               ('n_gen', 5),
               verbose=True,
               seed=1)

n_gen |  n_eval |   cv (min)   |   cv (avg)   |     fopt     |     favg    
    1 |       5 |  1.35027E+02 |  5.47570E+02 |            - |            -
    2 |      10 |  1.35027E+02 |  3.16525E+02 |            - |            -
    3 |      15 |  1.35027E+02 |  2.16131E+02 |            - |            -
    4 |      20 |  1.35027E+02 |  1.60240E+02 |            - |            -
    5 |      25 |  1.33435E+02 |  1.38575E+02 |            - |            -


In [6]:
res.X, res.F, res.G, res.CV

(None, None, None, None)

Here, the algorithm was not able to find any feasible solution in 5 generations. Therefore, all values contained in the results are equals to `None`. If in this case the least feasible solution should be returned, the flag `return_least_infeasible` needs to be enabled:

In [7]:
problem = get_problem("g01")
algorithm = GA(pop_size=5, return_least_infeasible=True)
res = minimize(problem,
               algorithm,
               ('n_gen', 5),
               verbose=True,
               seed=1)

n_gen |  n_eval |   cv (min)   |   cv (avg)   |     fopt     |     favg    
    1 |       5 |  1.35027E+02 |  5.47570E+02 |            - |            -
    2 |      10 |  1.35027E+02 |  3.16525E+02 |            - |            -
    3 |      15 |  1.35027E+02 |  2.16131E+02 |            - |            -
    4 |      20 |  1.35027E+02 |  1.60240E+02 |            - |            -
    5 |      25 |  1.33435E+02 |  1.38575E+02 |            - |            -


In [8]:
res.X, res.F, res.G, res.CV

(None, None, None, None)

We have made this design decision, because an infeasible solution can often not be considered as a solution
of the optimization problem. Therefore, having a solution equals to `None` indicates the fact no feasible solution has been found.

If the problem has multiple objective, the result object has the same structure but `res.X`, `res.F`, `res.G`, `res.CV` are a set 
of non-dominated solutions instead of a single one.

In [9]:
from pymoo.algorithms.nsga2 import NSGA2

problem = get_problem("zdt2")
algorithm = NSGA2()
res = minimize(problem,
               algorithm,
               ('n_gen', 10),
               seed=1)

In [10]:
res.F

array([[4.02723032e-01, 2.92285190e+00],
       [2.42936792e-03, 3.09049957e+00],
       [6.42640810e-05, 3.80452248e+00],
       [9.03843390e-02, 2.97779402e+00],
       [7.51601609e-04, 3.12489259e+00],
       [2.66638116e-05, 4.15122904e+00],
       [5.32367109e-01, 2.91333135e+00],
       [7.51074119e-05, 3.17766175e+00],
       [9.96726161e-01, 2.73633323e+00],
       [9.37668137e-01, 2.76323527e+00],
       [2.78148785e-05, 3.90537318e+00]])